Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of             4SI.JO   ABG.JO  ACL.JO  ACT.JO       ADH.JO  AEL.JO  \
Date                                                               
2005-06-01     NaN   8240.0  4980.0   130.0   133.701508     NaN   
2005-06-02     NaN   8250.0  4949.0   130.0   131.749695     NaN   
2005-06-03     NaN   8261.0  4949.0   130.0   131.749695     NaN   
2005-06-06     NaN   8290.0  5023.0   130.0   132.725601     NaN   
2005-06-07     NaN   8345.0  4845.0   149.0   131.749695     NaN   
...            ...      ...     ...     ...          ...     ...   
2025-05-26    70.0  16845.0   109.0   160.0  3199.000000  2350.0   
2025-05-27    70.0  16848.0   105.0   160.0  3210.000000  2350.0   
2025-05-28    73.0  17300.0   101.0   174.0  3283.000000  2355.0   
2025-05-29    70.0  17579.0   106.0   166.0  3247.000000  2362.0   
2025-05-30    70.0  17332.0   116.0   160.0  3215.000000  2358.0   

                  AFE.JO  AFH.JO  AFT.JO        AGL.JO  ...  VKE.JO   VOD.JO  \
Dat

In [4]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [5]:
df_SP500.info

<bound method DataFrame.info of                      A  AAL  AAP        AAPL        ABBV        ABNB  \
Date                                                                   
2005-06-01   17.296137  NaN  NaN    1.439286         NaN         NaN   
2005-06-02   17.303289  NaN  NaN    1.430000         NaN         NaN   
2005-06-03   17.210300  NaN  NaN    1.365714         NaN         NaN   
2005-06-06   17.539343  NaN  NaN    1.354286         NaN         NaN   
2005-06-07   17.324751  NaN  NaN    1.305000         NaN         NaN   
...                ...  ...  ...         ...         ...         ...   
2025-05-23  108.529999  NaN  NaN  195.270004  183.259995  126.720001   
2025-05-27  111.260002  NaN  NaN  200.210007  185.720001  129.399994   
2025-05-28  110.879997  NaN  NaN  200.419998  183.089996  128.669998   
2025-05-29  113.279999  NaN  NaN  199.949997  185.619995  128.360001   
2025-05-30  111.919998  NaN  NaN  200.850006  186.110001  129.000000   

                   ABT       AC

In [6]:
#Reduzierter SP500-Bestand ohne ausgeschiedene Unternehmen und auf Größe des JSE runtergebrochen

df_SP500_reduziert = pd.read_csv(
    'sp500_gefiltert_neuneu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    dayfirst=True
)

In [7]:
df_SP500_reduziert.info

<bound method DataFrame.info of                      A        AAPL         ABT        ADM         ADP  \
Date                                                                    
2005-06-01   17.296137    1.439286         NaN  20.100000   34.918613   
2005-06-02   17.303289    1.430000         NaN  20.430000   35.053432   
2005-06-03   17.210300    1.365714         NaN  20.219999   34.617249   
2005-06-06   17.539343    1.354286         NaN  20.150000   34.728275   
2005-06-07   17.324751    1.305000         NaN  20.690001   34.664829   
...                ...         ...         ...        ...         ...   
2025-05-23  108.529999  195.270004  131.300003  47.939999  321.089996   
2025-05-27  111.260002  200.210007  132.940002  48.630001  325.709991   
2025-05-28  110.879997  200.419998  132.020004  48.349998  324.140015   
2025-05-29  113.279999  199.949997  132.850006  48.369999  324.119995   
2025-05-30  111.919998  200.850006  133.580002  48.270000  325.529999   

                  

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [8]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [9]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [10]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [11]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [12]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [13]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [14]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [15]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [16]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [17]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [18]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [19]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [20]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [21]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [22]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [23]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [24]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [25]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [26]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [27]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [28]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [29]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [30]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [32]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [33]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [34]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [35]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [36]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [38]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [39]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [40]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [41]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [42]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [43]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [44]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [45]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [46]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [47]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [48]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [49]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [50]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [51]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [52]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

In [53]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [54]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    gewichte_equally_weighted = equally_weighted(training_daten)
    gewichte_min_variance= min_variance_opt(cov_matrix)
    gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": gewichte_equally_weighted,
    "Minimum-Variance": gewichte_min_variance,
    "Maximum-Diversification": gewichte_max_diversification,
    "Inverse-Volatility": gewichte_inverse_volatility,
    "Equal-Risk-Contribution": gewichte_risk_parity,
    "HRP": gewichte_hrp,
    "HERC": gewichte_herc
}
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df = pd.concat(portfolio_returns_list)
portfolio_returns_df = portfolio_returns_df.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.560718656371804e-05
            Iterations: 299
            Function evaluations: 78936
            Gradient evaluations: 299
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0661056745424897
            Iterations: 87
            Function evaluations: 22007
            Gradient evaluations: 83
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6806677955014559e-09
            Iterations: 21
            Function evaluations: 5544
            Gradient evaluations: 21
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6806473037277397e-05
            Iterations: 275
            Function evaluations: 72600
            Gradient evaluations: 275
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.920905052892258
            Iterations: 71
            Function evaluations: 18702
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4955482396956752e-09
            Iterations: 38
            Function evaluations: 10564
            Gradient evaluations: 38
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.646540406222078e-05
            Iterations: 371
            Function evaluations: 103510
            Gradient evaluations: 371
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9854150581059447
            Iterations: 84
            Function evaluations: 22412
            Gradient evaluations: 80
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6096719942643213e-09
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.101271457203561e-10
            Iterations: 66
            Function evaluations: 19206
            Gradient evaluations: 66
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.6933110341184316e-05
            Iterations: 272
            Function evaluations: 78881
            Gradient evaluations: 272
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5615330120689617
            Iterations: 36
            Function evaluations: 10474
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.409747818559429e-10
            Iterations: 66
            Function evaluations: 19140
            Gradient evaluations: 66
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.515

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00022394867292430918
            Iterations: 124
            Function evaluations: 36332
            Gradient evaluations: 124
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7955009315292858
            Iterations: 29
            Function evaluations: 8513
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.233255502579023e-10
            Iterations: 110
            Function evaluations: 32230
            Gradient evaluations: 110
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00022569483587998954
            Iterations: 113
            Function evaluations: 33335
            Gradient evaluations: 113
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3215314804758673
            Iterations: 42
            Function evaluations: 11931
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.969583184721256e-10
            Iterations: 67
            Function evaluations: 20971
            Gradient evaluations: 67
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.697985496953368e-05
            Iterations: 262
            Function evaluations: 82530
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3123645265484116
            Iterations: 40
            Function evaluations: 11380
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1373323212787251e-09
     

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.408001330022897e-10
            Iterations: 89
            Function evaluations: 37558
            Gradient evaluations: 89
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015734305955523813
            Iterations: 207
            Function evaluations: 87768
            Gradient evaluations: 207
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9858429498989154
            Iterations: 27
            Function evaluations: 11041
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.154457997399361e-10
            Iterations: 88
            Function evaluations: 37313
            Gradient evaluations: 88
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3453257309743086e-05
            Iterations: 373
            Function evaluations: 163374
            Gradient evaluations: 373
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.6220542053495226
            Iterations: 60
            Function evaluations: 24581
            Gradient evaluations: 56
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.823498102445649e-09
            Iterations: 38
            Function evaluations: 16644
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2570472725114772e-05
            Iterations: 335
            Function evaluations: 147065
            Gradient evaluations: 335
Optimization terminated successfully    (Exit mode 0)
            Current function value: 

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4037351740888413
            Iterations: 38
            Function evaluations: 17017
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2164028972934875e-09
            Iterations: 45
            Function evaluations: 20116
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.336828812328271e-05
            Iterations: 238
            Function evaluations: 106386
            Gradient evaluations: 238
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3472929859773752
            Iterations: 40
            Function evaluations: 16124
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0865034325044437e-09
   

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.696241554975139e-09
            Iterations: 37
            Function evaluations: 16910
            Gradient evaluations: 37
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7129267058505102e-05
            Iterations: 293
            Function evaluations: 134488
            Gradient evaluations: 293
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0425828665257693
            Iterations: 63
            Function evaluations: 27137
            Gradient evaluations: 59
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1195316375813742e-09
            Iterations: 38
            Function evaluations: 17442
            Gradient evaluations: 38
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4294372767715442e-05
            Iterations: 321
            Function evaluations: 149908
            Gradient evaluations: 321
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.357105474789429
            Iterations: 94
            Function evaluations: 42146
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.940671184612122e-09
            Iterations: 33
            Function evaluations: 15411
            Gradient evaluations: 33
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4384818409346062e-05
            Iterations: 382
            Function evaluations: 178013
            Gradient evaluations: 382
Optimization terminated successfully    (Exit mode 0)
            Current functio

In [55]:
#Exportieren nach CSV
portfolio_returns_df.to_csv("renditensp500_neu_f.csv")

## Ergebnis-Erzeugung JSE

In [56]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [57]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    #Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3134440810290774e-05
            Iterations: 152
            Function evaluations: 10184
            Gradient evaluations: 152
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.653137425667317
            Iterations: 53
            Function evaluations: 3590
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1202687138208137e-09
            Iterations: 44
            Function evaluations: 2948
            Gradient evaluations: 44
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.223441138083902e-05
            Iterations: 131
            Function evaluations: 8778
            Gradient evaluations: 131
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.235

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.7543156812255716
            Iterations: 36
            Function evaluations: 2666
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5276998741108466e-09
            Iterations: 42
            Function evaluations: 3067
            Gradient evaluations: 42
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3064556389717537e-05
            Iterations: 140
            Function evaluations: 10360
            Gradient evaluations: 140
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.381970760045858
            Iterations: 60
            Function evaluations: 4491
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.854170012627015e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.683014382006474e-09
            Iterations: 43
            Function evaluations: 3311
            Gradient evaluations: 43
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.9412981543266865e-05
            Iterations: 161
            Function evaluations: 12559
            Gradient evaluations: 161
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.84209872102518
            Iterations: 43
            Function evaluations: 3393
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.498560217486251e-09
            Iterations: 43
            Function evaluations: 3354
            Gradient evaluations: 43
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.49305789

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.826234078414551e-05
            Iterations: 142
            Function evaluations: 10792
            Gradient evaluations: 142
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0873662395432135
            Iterations: 37
            Function evaluations: 2845
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.25794543476294e-10
            Iterations: 99
            Function evaluations: 7525
            Gradient evaluations: 99
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.964045892800607e-05
            Iterations: 138
            Function evaluations: 10488
            Gradient evaluations: 138
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1134

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.50242489381647
            Iterations: 50
            Function evaluations: 4295
            Gradient evaluations: 50
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.49565840810712e-09
            Iterations: 68
            Function evaluations: 5780
            Gradient evaluations: 68
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6793775693611726e-05
            Iterations: 208
            Function evaluations: 17473
            Gradient evaluations: 208
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.544390252349358
            Iterations: 57
            Function evaluations: 4503
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.316071954319211e-09
            

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.129981686849365e-10
            Iterations: 93
            Function evaluations: 12834
            Gradient evaluations: 93
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.72416302357766e-05
            Iterations: 178
            Function evaluations: 24920
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8894599966729078
            Iterations: 77
            Function evaluations: 10353
            Gradient evaluations: 73
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2798217427026028e-08
            Iterations: 93
            Function evaluations: 13020
            Gradient evaluations: 93
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.0994

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5232543631874359e-05
            Iterations: 250
            Function evaluations: 37750
            Gradient evaluations: 250
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.269569646362189
            Iterations: 101
            Function evaluations: 14798
            Gradient evaluations: 97
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.814223470535556e-09
            Iterations: 61
            Function evaluations: 9212
            Gradient evaluations: 61
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3600606143243473e-05
            Iterations: 242
            Function evaluations: 36784
            Gradient evaluations: 242
Optimization terminated successfully    (Exit mode 0)
            Current function 

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.962019131541968
            Iterations: 88
            Function evaluations: 12810
            Gradient evaluations: 84
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.753787566968287e-09
            Iterations: 43
            Function evaluations: 6494
            Gradient evaluations: 43
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5527687349963467e-05
            Iterations: 220
            Function evaluations: 33440
            Gradient evaluations: 220
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.184237198969174
            Iterations: 107
            Function evaluations: 15841
            Gradient evaluations: 103
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.3596794764361764e-09
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.999862661400244e-09
            Iterations: 42
            Function evaluations: 6468
            Gradient evaluations: 42
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0638443984101544e-05
            Iterations: 262
            Function evaluations: 39824
            Gradient evaluations: 262
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.454284185655669
            Iterations: 98
            Function evaluations: 14444
            Gradient evaluations: 94
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.342262248551165e-09
            Iterations: 41
            Function evaluations: 6233
            Gradient evaluations: 41
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.090650657560652e-05
            Iterations: 274
            Function evaluations: 41648
            Gradient evaluations: 274
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.42072094048574
            Iterations: 124
            Function evaluations: 18525
            Gradient evaluations: 120
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.288669387680618e-09
            Iterations: 49
            Function evaluations: 7448
            Gradient evaluations: 49
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.979513492822373e-06
            Iterations: 273
            Function evaluations: 41770
            Gradient evaluations: 273
Optimization terminated successfully    (Exit mode 0)
            Current function va

In [58]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsJSE_neu_f.csv")

## Ledoit-Wolf Shrinkage

In [59]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

In [60]:
returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [61]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_gewichte_min_variance,
    "Maximum-Diversification": shrinked_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_gewichte_risk_parity,
    "HRP": shrinked_gewichte_hrp,
    "HERC": shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked = portfolio_returns_df_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5288862200654737e-05
            Iterations: 308
            Function evaluations: 81312
            Gradient evaluations: 308
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1767340633144645
            Iterations: 80
            Function evaluations: 20138
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.367469318835228e-09
            Iterations: 9
            Function evaluations: 2376
            Gradient evaluations: 9
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6505292460375198e-05
            Iterations: 293
            Function evaluations: 77352
            Gradient evaluations: 293
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.95

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0076215287471926
            Iterations: 79
            Function evaluations: 20930
            Gradient evaluations: 75
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.724209090209557e-09
            Iterations: 27
            Function evaluations: 7506
            Gradient evaluations: 27
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.621151098493346e-05
            Iterations: 295
            Function evaluations: 82306
            Gradient evaluations: 295
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.075824564769627
            Iterations: 64
            Function evaluations: 17641
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4618719221454564e-09
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1484385309851184e-09
            Iterations: 57
            Function evaluations: 16588
            Gradient evaluations: 57
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.878743690956813e-05
            Iterations: 246
            Function evaluations: 71340
            Gradient evaluations: 246
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.59186858309651
            Iterations: 36
            Function evaluations: 10477
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.080176662333151e-10
            Iterations: 66
            Function evaluations: 19140
            Gradient evaluations: 66
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.75481

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00022412743683273808
            Iterations: 106
            Function evaluations: 31058
            Gradient evaluations: 106
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8341259710400808
            Iterations: 29
            Function evaluations: 8516
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1828531031731546e-10
            Iterations: 110
            Function evaluations: 32230
            Gradient evaluations: 110
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00022545564587342235
            Iterations: 95
            Function evaluations: 28025
            Gradient evaluations: 95
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.35551475153263
            Iterations: 35
            Function evaluations: 10986
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.65405891768136e-10
            Iterations: 67
            Function evaluations: 20972
            Gradient evaluations: 67
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.6968829416809135e-05
            Iterations: 222
            Function evaluations: 69931
            Gradient evaluations: 222
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.342686556532796
            Iterations: 51
            Function evaluations: 14868
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.095078213397084e-09
         

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.05886249133464e-10
            Iterations: 80
            Function evaluations: 33761
            Gradient evaluations: 80
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015472366599622304
            Iterations: 193
            Function evaluations: 81832
            Gradient evaluations: 193
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0460674570198827
            Iterations: 28
            Function evaluations: 11895
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.568852663215338e-10
            Iterations: 89
            Function evaluations: 37736
            Gradient evaluations: 89
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.232869277941441e-05
            Iterations: 303
            Function evaluations: 132715
            Gradient evaluations: 303
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -3.724792290952102
            Iterations: 96
            Function evaluations: 40416
            Gradient evaluations: 92
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.354374957364167e-09
            Iterations: 39
            Function evaluations: 17082
            Gradient evaluations: 39
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1927390158292476e-05
            Iterations: 332
            Function evaluations: 145749
            Gradient evaluations: 332
Optimization terminated successfully    (Exit mode 0)
            Current function

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4273660198475295
            Iterations: 42
            Function evaluations: 17020
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1767978666191269e-09
            Iterations: 45
            Function evaluations: 20115
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.241555407363865e-05
            Iterations: 221
            Function evaluations: 98787
            Gradient evaluations: 221
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3736513283940344
            Iterations: 39
            Function evaluations: 15677
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0491657464190775e-09
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5161138361228742e-09
            Iterations: 38
            Function evaluations: 17367
            Gradient evaluations: 38
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6289281518743266e-05
            Iterations: 307
            Function evaluations: 140913
            Gradient evaluations: 307
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1008369745854174
            Iterations: 64
            Function evaluations: 27597
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9941818830023595e-09
            Iterations: 38
            Function evaluations: 17442
            Gradient evaluations: 38
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3830694044519194e-05
            Iterations: 306
            Function evaluations: 142902
            Gradient evaluations: 306
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.4353991787064535
            Iterations: 100
            Function evaluations: 44944
            Gradient evaluations: 96
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6403127585131448e-09
            Iterations: 20
            Function evaluations: 9340
            Gradient evaluations: 20
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3842053995495165e-05
            Iterations: 284
            Function evaluations: 132344
            Gradient evaluations: 284
Positive directional derivative for linesearch    (Exit mode 8)
            Cur

In [62]:
#Exportieren nach CSV
portfolio_returns_df_shrinked.to_csv("returnssp500shrinked_neu_f.csv")

### Auswertung JSE mit Ledoit Wolf

In [63]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [64]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.240047133151458e-05
            Iterations: 135
            Function evaluations: 9045
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.741709713501023
            Iterations: 43
            Function evaluations: 2923
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.635081070068458e-09
            Iterations: 44
            Function evaluations: 2949
            Gradient evaluations: 44
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.1064523258628265e-05
            Iterations: 127
            Function evaluations: 8509
            Gradient evaluations: 127
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.31063

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.860032031297268
            Iterations: 42
            Function evaluations: 3109
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1432680447431062e-09
            Iterations: 43
            Function evaluations: 3140
            Gradient evaluations: 43
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1022622534486556e-05
            Iterations: 114
            Function evaluations: 8436
            Gradient evaluations: 114
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.765161464093286
            Iterations: 57
            Function evaluations: 4285
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.933777645566643e-09
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.311800058952099e-09
            Iterations: 32
            Function evaluations: 2465
            Gradient evaluations: 32
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.700906009196339e-05
            Iterations: 143
            Function evaluations: 11154
            Gradient evaluations: 143
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9470234714474755
            Iterations: 49
            Function evaluations: 3869
            Gradient evaluations: 49
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0014079952513354e-09
            Iterations: 43
            Function evaluations: 3355
            Gradient evaluations: 43
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.051992

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.893064538128503e-05
            Iterations: 138
            Function evaluations: 10489
            Gradient evaluations: 138
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.156467487131176
            Iterations: 42
            Function evaluations: 3232
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.597926470366044e-10
            Iterations: 90
            Function evaluations: 6840
            Gradient evaluations: 90
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.112903238626374e-05
            Iterations: 141
            Function evaluations: 10716
            Gradient evaluations: 141
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1905

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.606103782804759
            Iterations: 52
            Function evaluations: 4468
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.690096735506445e-09
            Iterations: 56
            Function evaluations: 4760
            Gradient evaluations: 56
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.722853830987692e-05
            Iterations: 167
            Function evaluations: 14029
            Gradient evaluations: 167
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.6411501206122887
            Iterations: 52
            Function evaluations: 4417
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.403606949005002e-09
          

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.685769657042407e-10
            Iterations: 85
            Function evaluations: 11730
            Gradient evaluations: 85
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.650510459807652e-05
            Iterations: 132
            Function evaluations: 18480
            Gradient evaluations: 132
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.08693038450017
            Iterations: 69
            Function evaluations: 9736
            Gradient evaluations: 69
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0994257631782073e-08
            Iterations: 84
            Function evaluations: 11760
            Gradient evaluations: 84
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.958425

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5432432048360945e-05
            Iterations: 150
            Function evaluations: 22651
            Gradient evaluations: 150
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.566525391797201
            Iterations: 120
            Function evaluations: 18164
            Gradient evaluations: 119
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.662641786922386e-09
            Iterations: 51
            Function evaluations: 7701
            Gradient evaluations: 51
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4110048482834201e-05
            Iterations: 178
            Function evaluations: 27056
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.08311197335405
            Iterations: 84
            Function evaluations: 12648
            Gradient evaluations: 83
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3574733728141833e-09
            Iterations: 45
            Function evaluations: 6795
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4585913732774674e-05
            Iterations: 165
            Function evaluations: 25080
            Gradient evaluations: 165
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.317671964533073
            Iterations: 94
            Function evaluations: 13798
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0167320243649784e-09
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.536213275778372e-09
            Iterations: 33
            Function evaluations: 5083
            Gradient evaluations: 33
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1409462134752712e-05
            Iterations: 168
            Function evaluations: 25537
            Gradient evaluations: 168
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.519824718728864
            Iterations: 196
            Function evaluations: 30029
            Gradient evaluations: 192
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.131715040203033e-09
            Iterations: 29
            Function evaluations: 4408
            Gradient evaluations: 29
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2277

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1218483406704243e-05
            Iterations: 182
            Function evaluations: 27664
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.546489960492811
            Iterations: 117
            Function evaluations: 17817
            Gradient evaluations: 116
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.036273113821135e-09
            Iterations: 38
            Function evaluations: 5776
            Gradient evaluations: 38
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0595812918574394e-05
            Iterations: 204
            Function evaluations: 31212
            Gradient evaluations: 204
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6

In [65]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsJSEshrinked_neu_f.csv")

### Reduzierter S&P500-Bestand

In [66]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ADM       ADP      ADSK  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN  0.016418  0.003861       NaN   
2005-06-03 -0.005374 -0.044955       NaN -0.010279 -0.012443       NaN   
2005-06-06  0.019119 -0.008368       NaN -0.003462  0.003207       NaN   
2005-06-07 -0.012235 -0.036393       NaN  0.026799 -0.001827       NaN   
2005-06-08  0.009496  0.010399       NaN  0.014983 -0.014642       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.000417 -0.001741  0.001186   
2025-05-27  0.025154  0.025298  0.012490  0.014393  0.014388  0.016963   
2025-05-28 -0.003415  0.001049 -0.006920 -0.005758 -0.004820 -0.003762   
2025-05-29  0.021645 -0.002345  0.006287  0.000414 -0.000062 -0.007452   
2025-05-30 -0.012006  0.004501  0.005495 -0.002067  0.004350 -0.002963   

     

In [67]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_gewichte_min_variance,
    "Maximum-Diversification": reduziert_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_gewichte_risk_parity,
    "HRP": reduziert_gewichte_hrp,
    "HERC": reduziert_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert = portfolio_returns_df_reduziert.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9017992899290135e-05
            Iterations: 280
            Function evaluations: 29121
            Gradient evaluations: 280
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.652219442909011
            Iterations: 47
            Function evaluations: 4930
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.365055186761163e-09
            Iterations: 20
            Function evaluations: 2080
            Gradient evaluations: 20
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0021735577945475e-05
            Iterations: 257
            Function evaluations: 26728
            Gradient evaluations: 257
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.55

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6155153176558943
            Iterations: 49
            Function evaluations: 5682
            Gradient evaluations: 49
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8101393045322594e-09
            Iterations: 39
            Function evaluations: 4485
            Gradient evaluations: 39
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.906849094834389e-05
            Iterations: 274
            Function evaluations: 31510
            Gradient evaluations: 274
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6400833433064386
            Iterations: 48
            Function evaluations: 5565
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1923717473136432e-09
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.104976855330307e-09
            Iterations: 51
            Function evaluations: 6222
            Gradient evaluations: 51
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.9337898612931195e-05
            Iterations: 170
            Function evaluations: 20740
            Gradient evaluations: 170
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2711677774667747
            Iterations: 27
            Function evaluations: 3321
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1113257428256717e-09
            Iterations: 51
            Function evaluations: 6223
            Gradient evaluations: 51
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.86219

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00023194308735485647
            Iterations: 132
            Function evaluations: 16896
            Gradient evaluations: 132
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.715495262506066
            Iterations: 23
            Function evaluations: 2959
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3364525497407257e-10
            Iterations: 107
            Function evaluations: 13696
            Gradient evaluations: 107
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00023285493115072874
            Iterations: 124
            Function evaluations: 15873
            Gradient evaluations: 124
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.163895746368488
            Iterations: 27
            Function evaluations: 3913
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1948961345467531e-09
            Iterations: 62
            Function evaluations: 8928
            Gradient evaluations: 62
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.9534404206007455e-05
            Iterations: 212
            Function evaluations: 30528
            Gradient evaluations: 212
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1585972134560723
            Iterations: 25
            Function evaluations: 3626
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.49980288490844e-10
         

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.056435964745587e-10
            Iterations: 104
            Function evaluations: 20697
            Gradient evaluations: 104
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00016433063908191184
            Iterations: 176
            Function evaluations: 35025
            Gradient evaluations: 176
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9293340385245343
            Iterations: 21
            Function evaluations: 4196
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.364372727966718e-10
            Iterations: 105
            Function evaluations: 20895
            Gradient evaluations: 105
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6507380339393124e-05
            Iterations: 331
            Function evaluations: 68517
            Gradient evaluations: 331
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.3161844235497053
            Iterations: 56
            Function evaluations: 10813
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.896444314697936e-09
            Iterations: 52
            Function evaluations: 10764
            Gradient evaluations: 52
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5174083361355592e-05
            Iterations: 333
            Function evaluations: 68931
            Gradient evaluations: 333
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2281047356581687
            Iterations: 33
            Function evaluations: 6864
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.899582925166319e-10
            Iterations: 57
            Function evaluations: 11800
            Gradient evaluations: 57
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.614883644562528e-05
            Iterations: 224
            Function evaluations: 46368
            Gradient evaluations: 224
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1973374889750943
            Iterations: 31
            Function evaluations: 6446
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1865024100776278e-09
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6838674268885635e-09
            Iterations: 49
            Function evaluations: 10388
            Gradient evaluations: 49
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.106005586456038e-05
            Iterations: 254
            Function evaluations: 53848
            Gradient evaluations: 254
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8087680715598307
            Iterations: 47
            Function evaluations: 10004
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.025838224048078e-09
            Iterations: 38
            Function evaluations: 8057
            Gradient evaluations: 38
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8982

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7993405559428317e-05
            Iterations: 265
            Function evaluations: 56181
            Gradient evaluations: 265
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7373131331280534
            Iterations: 62
            Function evaluations: 12993
            Gradient evaluations: 61
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.800858914074732e-09
            Iterations: 27
            Function evaluations: 5725
            Gradient evaluations: 27
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8095451612740614e-05
            Iterations: 296
            Function evaluations: 62753
            Gradient evaluations: 296
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

In [68]:
#Exportieren nach CSV
portfolio_returns_df_reduziert.to_csv("returnsreduziert_neu_f.csv")

### Reduziert und Ledoit-Shrinkage

In [69]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ADM       ADP      ADSK  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN  0.016418  0.003861       NaN   
2005-06-03 -0.005374 -0.044955       NaN -0.010279 -0.012443       NaN   
2005-06-06  0.019119 -0.008368       NaN -0.003462  0.003207       NaN   
2005-06-07 -0.012235 -0.036393       NaN  0.026799 -0.001827       NaN   
2005-06-08  0.009496  0.010399       NaN  0.014983 -0.014642       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.000417 -0.001741  0.001186   
2025-05-27  0.025154  0.025298  0.012490  0.014393  0.014388  0.016963   
2025-05-28 -0.003415  0.001049 -0.006920 -0.005758 -0.004820 -0.003762   
2025-05-29  0.021645 -0.002345  0.006287  0.000414 -0.000062 -0.007452   
2025-05-30 -0.012006  0.004501  0.005495 -0.002067  0.004350 -0.002963   

     

In [70]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_shrinked_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_shrinked_gewichte_min_variance,
    "Maximum-Diversification": reduziert_shrinked_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_shrinked_gewichte_risk_parity,
    "HRP": reduziert_shrinked_gewichte_hrp,
    "HERC": reduziert_shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert_shrinked = portfolio_returns_df_reduziert_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8537208164551943e-05
            Iterations: 247
            Function evaluations: 25688
            Gradient evaluations: 247
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.739509480282772
            Iterations: 53
            Function evaluations: 5558
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.881345984878581e-09
            Iterations: 21
            Function evaluations: 2184
            Gradient evaluations: 21
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.958513194585269e-05
            Iterations: 244
            Function evaluations: 25376
            Gradient evaluations: 244
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.625

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.691647965059688
            Iterations: 50
            Function evaluations: 5801
            Gradient evaluations: 50
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6113527269492173e-09
            Iterations: 39
            Function evaluations: 4485
            Gradient evaluations: 39
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8793787664926943e-05
            Iterations: 254
            Function evaluations: 29210
            Gradient evaluations: 254
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.715673076932975
            Iterations: 57
            Function evaluations: 6611
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.852315334815508e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.029154585725517e-09
            Iterations: 51
            Function evaluations: 6223
            Gradient evaluations: 51
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.8476363060463746e-05
            Iterations: 162
            Function evaluations: 19764
            Gradient evaluations: 162
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3028576860097836
            Iterations: 29
            Function evaluations: 3566
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.03996591974305e-09
            Iterations: 51
            Function evaluations: 6222
            Gradient evaluations: 51
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.7791394

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000233094752151223
            Iterations: 95
            Function evaluations: 12160
            Gradient evaluations: 95
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7514355024500927
            Iterations: 26
            Function evaluations: 3340
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3943592277915643e-10
            Iterations: 99
            Function evaluations: 12673
            Gradient evaluations: 99
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00023355383184768086
            Iterations: 95
            Function evaluations: 12160
            Gradient evaluations: 95
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.74509

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.202820428381147
            Iterations: 30
            Function evaluations: 4348
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1300236268714454e-09
            Iterations: 62
            Function evaluations: 8928
            Gradient evaluations: 62
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.959486783546945e-05
            Iterations: 170
            Function evaluations: 24481
            Gradient evaluations: 170
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.198358752141978
            Iterations: 26
            Function evaluations: 3771
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.991320645731453e-10
          

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.499163357106975e-10
            Iterations: 105
            Function evaluations: 20895
            Gradient evaluations: 105
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00016140144614713875
            Iterations: 172
            Function evaluations: 34228
            Gradient evaluations: 172
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9798963947689674
            Iterations: 22
            Function evaluations: 4398
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.250689490614575e-10
            Iterations: 95
            Function evaluations: 18906
            Gradient evaluations: 95
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4845989613168498e-05
            Iterations: 269
            Function evaluations: 55684
            Gradient evaluations: 269
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.4302191924980767
            Iterations: 68
            Function evaluations: 13308
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.15677077267633e-09
            Iterations: 53
            Function evaluations: 10971
            Gradient evaluations: 53
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4434276347272143e-05
            Iterations: 300
            Function evaluations: 62100
            Gradient evaluations: 300
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2511647298801853
            Iterations: 34
            Function evaluations: 7070
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.656800834938903e-10
            Iterations: 58
            Function evaluations: 12006
            Gradient evaluations: 58
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.468819582104474e-05
            Iterations: 222
            Function evaluations: 45954
            Gradient evaluations: 222
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.221022217826816
            Iterations: 29
            Function evaluations: 6032
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1415111194283855e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0181479476007934e-09
            Iterations: 38
            Function evaluations: 8056
            Gradient evaluations: 38
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9970406724244605e-05
            Iterations: 252
            Function evaluations: 53425
            Gradient evaluations: 252
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8549916478006305
            Iterations: 43
            Function evaluations: 9156
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8931966784702504e-09
            Iterations: 39
            Function evaluations: 8268
            Gradient evaluations: 39
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7944

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7007507461161723e-05
            Iterations: 259
            Function evaluations: 54909
            Gradient evaluations: 259
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.837883415102487
            Iterations: 72
            Function evaluations: 15335
            Gradient evaluations: 72
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2787455436146e-09
            Iterations: 28
            Function evaluations: 5936
            Gradient evaluations: 28
2024-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.724010924601169e-05
            Iterations: 261
            Function evaluations: 55332
            Gradient evaluations: 261
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8174

In [71]:
#Exportieren nach CSV
portfolio_returns_df_reduziert_shrinked.to_csv("returnsreduziertshrinked_neu_f.csv")